# 2_train_model.ipynb

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import concatenate
from keras.utils import plot_model
from keras import regularizers
import pickle

## Load Data

In [ ]:
# load data
with open('data/all_data.pkl', 'rb') as file:
    price_series, time_categories, up_targets, dn_targets = pickle.load(file) 

# test inputs
# train_x = np.random.rand(1,20,10)
# train_y = np.random.rand(1,3)

timesteps = price_series.shape[1]
channels = price_series.shape[2]

## Split Data

In [ ]:
RANDOM_SEED = 7
TARGETS = up_targets

# Splitting the arrays into train and test sets
train_x_price, test_x_price = train_test_split(price_series, test_size=0.2, random_state=RANDOM_SEED)
train_x_time, test_x_time = train_test_split(time_categories, test_size=0.2, random_state=RANDOM_SEED)
train_y, test_y = train_test_split(TARGETS, test_size=0.2, random_state=RANDOM_SEED)

## Define Multimodal Convolutional Network

In [ ]:
FILTERS = 64

input_cnn = Input(shape=(timesteps,channels,))
cnn = Conv1D(filters=FILTERS, kernel_size=3, padding='same', 
                 activation='relu', data_format='channels_last',
                 activity_regularizer=None,
                 input_shape=(timesteps, channels))(input_cnn)

cnn = Conv1D(filters=FILTERS, kernel_size=2, padding='same', strides=2, 
                 activation=None, use_bias=False, data_format='channels_last', 
                 activity_regularizer=None)(cnn)

cnn = Conv1D(filters=FILTERS*2, kernel_size=3, padding='same', 
                 activation='relu', data_format='channels_last', 
                 activity_regularizer=None)(cnn)

cnn = Conv1D(filters=FILTERS*2, kernel_size=2, padding='same', strides=2, 
                 activation=None, use_bias=False, data_format='channels_last',
                 activity_regularizer=None)(cnn)

cnn = Conv1D(filters=FILTERS*4, kernel_size=3, padding='same', 
                 activation='relu', data_format='channels_last', 
                 activity_regularizer=None)(cnn)

cnn = Dropout(0.0)(cnn)
cnn = Flatten()(cnn)
cnn = Model(inputs=input_cnn, outputs=cnn)

# perceptron branch
input_mlp = Input(shape=(38,))
mlp = Dense(8, activation='relu', activity_regularizer=regularizers.L2(0.01))(input_mlp)
mlp = Model(inputs=input_mlp, outputs=mlp)

# join branches
combined = concatenate([cnn.output, mlp.output])
head = Dense(512, activation='relu')(combined)
head = Dense(128, activation='relu')(head)
head = Dense(3, activation='softmax')(head)
model = Model(inputs=[cnn.input, mlp.input], outputs=head)

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## Train And Evaluate Model

In [ ]:
EPOCHS = 10
BATCH_SIZE = 32

model.fit([train_x_price, train_x_time], train_y, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=False)

_, accuracy = model.evaluate([test_x_price, test_x_time], test_y, batch_size=BATCH_SIZE, verbose=0)
print(accuracy)

## Evaluate Precision-Recall Curves

### Notes

In [ ]:
## add cross validation
## pad - most recent edge is important!
## replace maxpooling with dilated kernel to preserve data - geoff hinton
## replace maxpooling with stride = 2 - 'learnable pooling, all-convolution network'
## batch norm on conv layers?
## groups = channels maintains depthwise separation.  Add same multiple of filters?
## l2 less sensitive to small changes (noise), l1 blocks some inputs
## layers.DepthwiseConv1D() reduces number of parameters
## model.train(sample_weight=) heavier recent samples